In [3]:
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import torch.optim as optim
import torch.optim.lr_scheduler as lr_sched
import torch.utils.data as data

In [ ]:
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

In [ ]:
"""
Do grzyb/nie grzyb - transform_raw_1
Do klasyfikacji - transform_raw_2 (zachowuje więcej szczegółów)
"""
transform_raw_1 = transforms.Compose(
    transforms.Resize(96, 96),
    transforms.ToTensor(),
     transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
)

transform_raw_2 = transforms.Compose(
    transforms.Resize(250, 250),
    transforms.ToTensor(),
     transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
)

In [ ]:
dataset = torchvision.datasets.ImageFolder("DS_MASHROOM_OR_NOT", transform=transform_raw_1)
len(dataset)

In [ ]:
seed = 123
torch.manual_seed(seed)

In [ ]:
train_size = 50000
val_size = len(dataset) - train_size

train_ds, val_ds = data.random_split(dataset, [train_size, val_size])

In [ ]:
batch_size = 500

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, pin_memory=True, num_workers=1)
val_loader = DataLoader(val_ds, batch_size=batch_size, pin_memory=True, num_workers=1)

In [ ]:
class GoogleNet_Block(nn.Module):
    """
    in_channels - num. channles into block
    out_channels - num. concatenated channles going out from block
    conv_size_in - list of num. of channels into 1,3 and 5 conv (in that order)
    conv_size_out - list of num. of channels going out from 1,3,5 conv (-||-)
    dim_redu - dimention reduction before 3 and 5 conv + after 3x3 pool (not obligatory)
    """

    def __init__(self, in_channels, out_channels, conv_size_in:list, conv_size_out:list, stride:list=[1,1,2], padding:list=[0, 1, 2], dim_redu:list=[]):
        super().__init__()

        # conv 1x1
        self.conv1 = nn.Conv2d(conv_size_in[0], conv_size_out[0], stride=stride[0], padding=padding[0])

